In [1]:
%reset -f
import glob, os, sys, io
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
import keras

from wavhandler import *
from utils import *

import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)
import seaborn as sns
sns.set()

import librosa
from scipy import signal
import math
seed = 2018
np.random.seed(seed)
from matplotlib import pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.layers import Input
from keras import layers

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger

from keras import Model
from keras import backend as K

from keras.utils import np_utils
from keras.preprocessing import image

from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception

Using TensorFlow backend.


In [2]:
target_names = all_6
current_model = DenseNet121
model_name = './data/wingbeats_' + current_model.__name__

best_weights_path = model_name + '.h5'
log_path = model_name + '.log'
monitor = 'val_acc'
batch_size = 32
epochs = 100
es_patience = 7
rlr_patience = 3

SR = 8000
N_FFT = 256
HOP_LEN = int(N_FFT / 6)
input_shape = (129, 120, 1)

img_input = Input(shape = input_shape)
pretrained = current_model(input_tensor = img_input, 
                           classes = len(target_names), 
                           weights = best_weights_path)

In [3]:
bi_classes = ['LG_drosophila_10_09', 'LG_zapr_26_09']
target_names = bi_classes
X_names,y = get_data(target_names=bi_classes, nr_signals=999999, only_names=True)

# X_names, y = shuffle(X_names, y, random_state = seed)
# X_train, X_test, y_train, y_test = train_test_split(X_names, y, stratify = y, test_size = 0.20, random_state = seed)

100%|██████████| 2/2 [00:00<00:00, 34.61it/s]


In [4]:
def predict_generator():
    while True:
        for start in range(0, len(X_names), batch_size):
            x_batch = []
            y_batch = []

            end = min(start + batch_size, len(X_names))
            test_batch = X_names[start:end]
            labels_batch = y[start:end]

            for i in range(len(test_batch)):
                data, rate = librosa.load(test_batch[i], sr = SR)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
#                 data = librosa.amplitude_to_db(data)
                data = np.abs(data)
                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)

            #y_batch = np_utils.to_categorical(y_batch, len(target_names))

            yield x_batch, y_batch

In [5]:
pretrained.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# loss, acc = pretrained.evaluate_generator(valid_generator(),
#         steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

In [7]:
X = pretrained.predict_generator(predict_generator(),
         steps = int(math.ceil(float(len(X_names)) / float(batch_size))),
                                verbose=1,
                                workers=8)

399/399 [==============================] - 82s 205ms/step


In [8]:
X.shape

(12746, 6)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=3)

"""
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=0)
ros.fit(X,y)
X, y = ros.fit_resample(X,y)
print('After undersampling: \n{}\n'.format(pd.DataFrame(y).iloc[:,0].value_counts()))
"""

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

classifier = xgboost.XGBClassifier(n_estimators=100, learning_rate=0.2, n_jobs=-1)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

ac = accuracy_score(y_test, y_pred)
cv_ac = cross_val_score(classifier, X, y, cv=5, scoring='accuracy')
print("Name: %s, ac: %f" % ('XGBoost', ac))
print("Name: %s, cv_ac: %f" % ('XGBoost', np.mean(cv_ac)))

Name: XGBoost, ac: 0.794582
Name: XGBoost, cv_ac: 0.812047
